In [1]:
import sys
import ROOT as R
# R.EnableImplicitMT()
%jsroot on

In [2]:
R.gSystem.Load("/data/HPS/lib/libMoller.dylib")
Moller = R.Moller()
print("Version of Moller Class:",Moller.GetVersion())

Version of Moller Class: 0.3.0


In [3]:
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/rafo_minidst/hps_007966.?_Moller_4.2_minidst.root"
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/hps_007966.n_Moller_4.2.root"
#snapshot_file_name = "moller_rafo_filtered.root"
data_files_pattern = "/data/HPS/data/physrun2016/Moller/pass4kf/hps_007966.#_recon_5.2.1_minidst.root"
#snapshot_file_name = "moller_pass4kf_filtered.root"
ch_moller = R.TChain("MiniDST")
nf=0
for i in range(10):
    nf += ch_moller.Add(data_files_pattern.replace("#",str(i)))
# nf = ch_moller.Add(data_files_pattern)
n_events = ch_moller.GetEntries()
df_moller = R.RDataFrame(ch_moller)
df = df_moller
print(f"Added {nf} files to the chain, total of {n_events/1e6:7.2f}M events")

Added 10 files to the chain, total of    4.72M events


In [4]:
Particle_names=["Final state particle KF", "UC V0 vertex KF", "BSC V0 vertex KF", "TC V0 vertex KF", "UC Moller vertex KF",
"BSC Moller vertex KF", "TC Moller vertex KF", "Other electron KF", "UC VC vertex KF", "Final state particle GBL", "UC V0 vertex GBL",
"BSC V0 vertex GBL", "TC V0 vertex GBL", "UC Moller vertex", "BSC Moller vertex", "TC Moller vertex", "Other electron GBL",
"UC VC vertex GBL"]
All_Names = list(df.GetColumnNames())
Store_Names = []
ii=0
remove_names = [] # ["v0_","hodo_","ext_trigger","part_","rf_","svt"]
for n in All_Names:
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):
        Store_Names.append(s)
        dat_type = df.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_nhits             [RVec<int>           ] ecal_cluster_seed_energy       [RVec<double>        ] 
ecal_cluster_seed_index        [RVec<int>           ] ecal_cluster_seed_ix           [RVec<int>           ] 
ecal_cluster_seed_iy           [RVec<int>           ] ecal_cluster_time              [RVec<double>        ] 
ecal_cluster_uncor_energy      [RVec<double>        ] ecal_cluster_uncor_hits        [RVec<vector<int>>   ] 
ecal_cluster_uncor_nhits       [RVec<int>           ] ecal_cluster_uncor_seed_energy [RVec<double>        ] 
ecal_cluster_uncor_seed_index  [RVec<int>           ] ecal_cluster_uncor_seed_ix     [RVec<int>           ] 
ecal_cluster_uncor_seed_iy     [RVec<int>           ] ecal_cluster_uncor_time        [RVec<double>        ] 
ecal_cluster_uncor_x           [RVec<double>        ] ecal_cluster_uncor_y           [RVec<double>        ] 
ecal_cluster_uncor_

In [5]:
dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df), 0., 2.,"electron_pairs")
dfx2 = Moller.Select_El_Pairs(R.RDF.AsRNode(df), 0., 200.,"electron_pairs")
dfx2 = dfx2.Define("p4v1",""" 
RVec<TLorentzVector> out;
      for(int i=0; i<electron_pairs.size(); ++i) {
         TLorentzVector v4;
         int item = electron_pairs[i].first;
         v4.SetXYZM(part_px[item], part_py[item], part_pz[item], 0.000510998949996);
         v4.RotateY(-0.0302);
         out.emplace_back(v4);
      }
      return out;""") \
    .Define("p4v2",""" 
RVec<TLorentzVector> out;
      for(int i=0; i<electron_pairs.size(); ++i) {
         TLorentzVector v4;
         int item = electron_pairs[i].second;
         v4.SetXYZM(part_px[item], part_py[item], part_pz[item], 0.000510998949996);
         v4.RotateY(-0.0302);
         out.emplace_back(v4);
      }
      return out;""")

In [6]:
#h_ee_time = dfx2.Define("e_delta_time","vector<double> out; for(int i=0; i<electron_pairs.size(); ++i){int i1= part_track[electron_pairs[i].first];int i2= part_track[electron_pairs[i].second]; out.push_back(track_time[i1]-track_time[i2]);} return out;").Histo1D(("h_ee_time","e- time difference",500,-50.,50.),"e_delta_time")
dfx2 = dfx2.Define("e_delta_time","vector<double> out; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){ int i1= part_track[electron_pairs[i].first];int i2= part_track[electron_pairs[i].second]; if(part_ecal_cluster[electron_pairs[i].first]<0){int tmp=i2; i2=i1; i1=tmp;}; out.push_back(track_time[i1]-track_time[i2]);}}; return out;").Filter("e_delta_time.size()>0")
h_ee_time2 = dfx2.Histo1D(("h_ee_time","e- track time difference track with ecal cluster first;#Delta t [ns]",500,-20.,20.),"e_delta_time")
h_ee_time3 = dfx2.Define("e_delta_time3","vector<double> out; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){int i1= part_track[electron_pairs[i].first];int i2= part_track[electron_pairs[i].second];if(part_ecal_cluster[electron_pairs[i].first]<0){int tmp=i2; i2=i1; i1=tmp;}; out.push_back(track_time[i1]-track_time[i2]);}}; return out;").Histo1D(("h_ee_time","#Delta t tracks;#Delta t [ns]",500,-20.,20.),"e_delta_time3")
h_n_e_with_ecal = dfx2.Define("n","int out=0; for(int i=0; i< part_pdg.size(); ++i){if( part_pdg[i] == 11 && part_ecal_cluster[i]>=0){out++;}}; return out;").Histo1D(("h_n_e_with_ecal","Number of e- with ecal cluster",11,-0.5,10.5),"n")
h_n_ee = dfx2.Define("n","int out=0; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){++out;}}; return out;").Histo1D(("h_n_ee","Number of Moller pairs",11,-0.5,10.5),"n")
h_n_ee_with_0ecal = dfx2.Define("n","int out=0; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); int q=0; if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){ if(part_ecal_cluster[electron_pairs[i].first]>=0){++q;}; if(part_ecal_cluster[electron_pairs[i].second]>=0){++q;}; if(q==0){++out;}}}; return out;").Histo1D(("h_n_ee_with_1ecal","Number of e- with 0 ecal clusters",11,-0.5,10.5),"n")
h_n_ee_with_1ecal = dfx2.Define("n","int out=0; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); int q=0; if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){if(part_ecal_cluster[electron_pairs[i].first]>=0){++q;}; if(part_ecal_cluster[electron_pairs[i].second]>=0){++q;}; if(q==1){++out;}}}; return out;").Histo1D(("h_n_ee_with_1ecal","Number of e- with 1 ecal cluster",11,-0.5,10.5),"n")
h_n_ee_with_2ecal = dfx2.Define("n","int out=0; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); int q=0; if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){if(part_ecal_cluster[electron_pairs[i].first]>=0){++q;}; if(part_ecal_cluster[electron_pairs[i].second]>=0){++q;}; if(q==2){++out;}}}; return out;").Histo1D(("h_n_ee_with_2ecal","Number of e- with 1 ecal cluster",11,-0.5,10.5),"n")

In [7]:
cc0a = R.TCanvas("cc0a","cc0a",1200,400)
cc0a.Divide(2,1)
pad1 = cc0a.cd(1)
pad1.SetLogy()
h_n_ee_with_2ecal.SetStats(0)
h_n_ee_with_2ecal.Draw()
h_n_e_with_ecal.Draw("same")
h_n_ee.SetLineColor(R.kCyan)
h_n_ee.Draw("same")
h_n_ee_with_1ecal.SetLineColor(R.kBlue)
h_n_ee_with_1ecal.SetLineWidth(3)
h_n_ee_with_1ecal.Draw("same")
h_n_ee_with_2ecal.SetLineColor(R.kGreen)
h_n_ee_with_2ecal.SetLineWidth(2)
h_n_ee_with_2ecal.Draw("same")
h_n_ee_with_0ecal.SetLineColor(R.kRed)
h_n_ee_with_0ecal.SetLineWidth(1)
h_n_ee_with_0ecal.Draw("same")

legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_n_e_with_ecal.GetPtr(),"e- with ecal","l")
legend.AddEntry(h_n_ee.GetPtr(),"All Moller pairs","l")
legend.AddEntry(h_n_ee_with_0ecal.GetPtr(),".. with 0 ecal clusters","l")
legend.AddEntry(h_n_ee_with_1ecal.GetPtr(),".. with 1 ecal cluster","l")
legend.AddEntry(h_n_ee_with_2ecal.GetPtr(),".. with 2 ecal clusters","l")
legend.Draw()
#h_ee_time.Draw()
pad2 = cc0a.cd(2)
h_ee_time2.SetLineColor(R.kBlue)
h_ee_time2.Draw("same")
h_ee_time3.SetLineColor(R.kRed)
h_ee_time3.Draw("same")

cc0a.Draw()

In [16]:
disp = dfx2.Define("e_delta_time","vector<double> out; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){int i1= part_track[electron_pairs[i].first];int i2= part_track[electron_pairs[i].second]; out.push_back(track_time[i1]-track_time[i2]);}}; return out;").Define("ee_pairs1","vector< pair<int,int> > out; for(int i=0; i<electron_pairs.size(); ++i){auto mv4=(p4v1[i]+p4v2[i]); if(mv4.P() >2.1 && mv4.P() < 2.45 && mv4.Theta() < 0.01){ out.push_back(electron_pairs[i]);}}; return out;").Filter("e_delta_time.size()>0").Define("part_time","vector<double> out; for(int i=0;i<part_pdg.size();i++){if(part_track[i]>=0){out.push_back(track_time[part_track[i]]);}else{out.push_back(-999.);}}; return out;").Display(("ee_pairs1","part_pdg","part_ecal_cluster","part_time","e_delta_time"),25,10)
disp.Print()

+------+-----------+----------+-------------------+-------------+--------------+
| Row  | ee_pairs1 | part_pdg | part_ecal_cluster | part_time   | e_delta_time | 
+------+-----------+----------+-------------------+-------------+--------------+
| 62   | { 1, 2 }  | 11       | -1                | -30.475922  | 65.734212    | 
|      |           | 11       | 1                 | 0.924390    |              | 
|      |           | 11       | -1                | -64.809822  |              | 
|      |           | 11       | -1                | 31.326269   |              | 
|      |           | 22       | 0                 | -999.000000 |              | 
|      |           | 22       | 2                 | -999.000000 |              | 
+------+-----------+----------+-------------------+-------------+--------------+
| 90   | { 0, 1 }  | 11       | -1                | 0.598526    | 3.489876     | 
|      |           | 11       | 0                 | -2.891349   |              | 
|      |           

In [4]:
#dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df),2.)
#df = df.Define("track_theta","RVec<double> out; for(int i=0;i<track_px.size();++i){out.push_back(atan2(sqrt(track_px[i]*track_px[i]+track_py[i]*track_py[i]),track_pz[i]));} return out;")
dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df), 0., 2.,"electron_pairs")    # Select delta time < 2ns.
dfxx = Moller.Select_El_Pairs(R.RDF.AsRNode(df),4., 20., "electron_pairs")  # Select delta time > 4ns and < 20ns.

# The cut for 2 and only 2 electrons is similar to the one done for the predefined Moller pairs, but only serves to cut the statistics. Uncomment below to get the
# statistics similar for the Beamspot constrained Moller pairs.
#dfx = dfx.Define("n_emin","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Filter("n_emin == 2").Filter("electron_pairs.size() == 1")
#dfxx = dfxx.Define("n_emin","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Filter("n_emin == 2").Filter("electron_pairs.size() == 1")
d = [dfx, dfxx]
for i in range(len(d)):
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "electron_pairs","part_","p4")
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 2.306*0.75, 2.306*1.15, "electron_pairs", "el_pairs_r1")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r1","part_","r1_p4");
    d[i] = Moller.Refine_El_Pairs_2(R.RDF.AsRNode(d[i]), "el_pairs_r1", "el_pairs_r2")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r2","part_","r2_p4");
    d[i] = Moller.Refine_El_Pairs_3(R.RDF.AsRNode(d[i]), 0.01, "el_pairs_r2", "r2_", "el_pairs_r3")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r3","part_","r3_p4");
    
    # These cuts should not be done.
    d[i] = Moller.Refine_El_Pairs_X(R.RDF.AsRNode(d[i]), 0.040, 0.048, "r2_p4tht1", "r2_p4tht2" ,"el_pairs_r2", "el_pairs_rx")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_rx","part_","rx_p4");
    d[i] = d[i].Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")

# Collection of indexes to ALL the electron particles and all the electron tracks
#     d[i] = d[i].Define("e_part_idx","vector<int> out;for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out.push_back(i);} return out;")\
#    .Define("e_track_idx","vector<int> out;for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out.push_back(part_track[i]);} return out;")
# Collection of indexes to the electron particles of the Moller pair
#     d[i] = d[i].Define("pair_e_part_idx","std::set<int> out;for(int i=0; i<electron_pairs.size(); ++i){out.insert(electron_pairs[i].first);out.insert(electron_pairs[i].second);} return out;")

dfx, dfxx = d

In [6]:
num = dfx.Count()
num2 = dfxx.Count()
print(f" Num = {num.GetValue()}   Num2 = {num2.GetValue()}")

 Num = 397975   Num2 = 1276230


In [7]:
 # Example - Check that the charge of electrons is -1, first with indirection, then with a direct selection on the part_pdg.
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<e_part_idx.size();++i){out.push_back(part_charge[e_part_idx[i]]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<part_charge.size();++i){if(part_pdg[i]==11) out.push_back(part_charge[i]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
h_n_part = dfx.Define("n_part","return part_pdg.size()").Histo1D(("h_n_part","Number of particles",11,-0.5,10.5),"n_part")
h_n_neg_track = dfx.Define("n_neg_track","int count=0;for(int i=0;i<track_omega.size();++i){if(track_omega[i]>0 && track_type[i]==1)count++;} return count;").Histo1D(("h_n_neg_track","Number of - tracks",11,-0.5,10.5),"n_neg_track")
h_n_electron = dfx.Define("n_electron","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Histo1D(("h_n_electron","Number of e-",11,-0.5,10.5),"n_electron")
h_n_moller = dfx.Define("n_moller","return electron_pairs.size()").Histo1D(("h_n_moller","Number of Moller pairs",11,-0.5,10.5),"n_moller")

h_track_time_diff = dfx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("h_track_time_diff","track_time_diff",500,-50.,50.),"track_time_diff")
hx_track_time_diff = dfxx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hx_track_time_diff","track_time_diff",500,-50.,50.),"track_time_diff")

h_track_chi2 = dfx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("h_track_chi2","track_chi2",500,0.,10.),"t_chi2")
hx_track_chi2 = dfxx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("xh_track_chi2","track_chi2",500,0.,10.),"t_chi2")

h_track_nhit = dfx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("h_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")
hx_track_nhit = dfxx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("hx_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")

h_beam_theta_x = dfx.Histo1D(("h_beam_theta_x","Beam Angle #theta_{x}",1000,-0.1,0.1),"p4beam_theta_x")
h_beam_theta_x_r2 = dfx.Filter("el_pairs_r2.size()>=1").Histo1D(("h_beam_theta_x_r2","Beam Angle #theta_{x}",1000,-0.1,0.1),"r2_p4beam_theta_x")
h_beam_theta_x_r3 = dfx.Filter("el_pairs_r3.size()>=1").Histo1D(("h_beam_theta_x_r3","Beam Angle #theta_{x}",1000,-0.1,0.1),"r3_p4beam_theta_x")
hx_beam_theta_x = dfxx.Histo1D(("hx_beam_theta_x","Beam Angle #theta_{x}",1000,-0.1,0.1),"p4beam_theta_x")

In [8]:
%jsroot on
cc0 = R.TCanvas("cc0","cc0",1200,1200)
cc0.Divide(2,3)
cc0.cd(1)
h_n_moller.SetTitle("Particle counts")
h_n_moller.SetStats(0)
h_n_moller.SetLineWidth(2)
h_n_moller.SetLineColor(R.kCyan)
h_n_moller.Draw("same")
h_n_electron.SetLineWidth(3)
h_n_electron.SetLineColor(R.kBlue)
h_n_electron.Draw("same")
h_n_neg_track.SetLineColor(R.kRed)
h_n_neg_track.Draw("same")
h_n_part.SetLineWidth(1)
h_n_part.SetLineColor(R.kGreen+2)
h_n_part.Draw("same")
legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_n_part.GetPtr(),"any particle","l")
legend.AddEntry(h_n_neg_track.GetPtr(),"neg. tracks","l")
legend.AddEntry(h_n_electron.GetPtr(),"electron","l")
legend.AddEntry(h_n_moller.GetPtr(),"Moller","l")
legend.Draw()
cc0.cd(2)
h_track_time_diff.SetLineWidth(1)
h_track_time_diff.SetLineColor(R.kGreen+2)
h_track_time_diff.Draw()
hx_track_time_diff.SetLineColor(R.kRed)
hx_track_time_diff.Draw("same")
cc0.cd(3)
h_track_chi2.SetLineWidth(1)
h_track_chi2.SetLineColor(R.kGreen+2)
h_track_chi2.Draw()
hx_track_chi2.SetLineColor(R.kRed)
hx_track_chi2.Draw("same")
cc0.cd(4)
hx_track_nhit.SetStats(0)
hx_track_nhit.SetLineWidth(1)
hx_track_nhit.SetLineColor(R.kGreen+2)
hx_track_nhit.Draw("same")
h_track_nhit.SetStats(0)
h_track_nhit.SetLineColor(R.kRed)
h_track_nhit.Draw("same")

pad5 = cc0.cd(5)
pad5.SetLogy()
hx_beam_theta_x.SetStats(0)
hx_beam_theta_x.SetLineColor(R.kRed)
hx_beam_theta_x.Draw("same")
#hx_beam_theta_x.Fit("gaus")

h_beam_theta_x.SetStats(0)
h_beam_theta_x.SetLineWidth(1)
h_beam_theta_x.SetLineColor(R.kGreen+2)
h_beam_theta_x.Draw("same")
#h_beam_theta_x.Fit("gaus")

h_beam_theta_x_r2.SetStats(0)
h_beam_theta_x_r2.SetLineWidth(1)
h_beam_theta_x_r2.SetLineColor(R.kBlue)
h_beam_theta_x_r2.Draw("same")
#h_beam_theta_x_r2.Fit("gaus")

h_beam_theta_x_r3.SetStats(0)
h_beam_theta_x_r3.SetLineWidth(1)
h_beam_theta_x_r3.SetLineColor(R.kOrange)
h_beam_theta_x_r3.Draw("same")
#h_beam_theta_x_r3.Fit("gaus")

leg2 = R.TLegend(0.7,0.7,0.9,0.9)
leg2.AddEntry(hx_beam_theta_x.GetPtr(),"Not Moller","l")
leg2.AddEntry(h_beam_theta_x.GetPtr(),"Moller","l")
leg2.AddEntry(h_beam_theta_x_r2.GetPtr(),"Moller r2","l")
leg2.AddEntry(h_beam_theta_x_r3.GetPtr(),"Moller r3","l")
leg2.Draw()

cc0.Draw()

In [9]:
h_v0_type = dfx.Histo1D(("h_v0_type","V0 Type",17,-0.5,16.5),"v0_type")
#("h_n_v0","n_v0",11,-0.5,10.5)
cc1 = R.TCanvas("cc1","cc1",1000,500)
#cc1.Divide(1,2)
pad1= cc1.cd(1)
pad1.SetBottomMargin(0.2)
axis = h_v0_type.GetXaxis()
axis.SetLabelSize(0.035)
for i in range (0,16):
    axis.SetBinLabel(i+1,Particle_names[i])
h_v0_type.SetStats(0)
h_v0_type.SetFillColor(R.kBlue-10)
h_v0_type.Draw()
cc1.Draw()

In [10]:
h_v0_mass_UC = dfx.Define("v0_mass_UC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==13) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_UC","v0 mass UC; M_{inv} [GeV]",500,0.,0.2),"v0_mass_UC")
h_v0_mass_BSC = dfx.Define("v0_mass_BSC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==14) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_BSC","v0 mass UCM_{inv} [GeV]",500,0.,0.2),"v0_mass_BSC")
h_v0_mass_TC = dfx.Define("v0_mass_TC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==15) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_TC","v0 mass TCM_{inv} [GeV]",500,0.,0.2),"v0_mass_TC")

In [11]:
h_ee_mass_r2 = dfx.Histo1D(("h_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")
hx_ee_mass_r2 = dfxx.Histo1D(("hx_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")
h_ee_mass_r3 = dfx.Histo1D(("h_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")
hx_ee_mass_r3 = dfxx.Histo1D(("hx_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")

h_ee_mass_rx = dfx.Histo1D(("h_ee_mass_rx","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx_p4minv")
hx_ee_mass_rx = dfxx.Histo1D(("hx_ee_mass_rx","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx_p4minv")
h_ee_thtE1 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht1","p4E1")
h_ee_thtE2 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht2","p4E2")
h_ee_thth = dfx.Histo2D(("h_ee_thth","EE theta-theta;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"p4tht1","p4tht2")
h_ee_thtE1_r1 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1 R1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r1_p4tht1","r1_p4E1")
h_ee_thtE2_r1 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2 R1;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r1_p4tht2","r1_p4E2")
h_ee_thth_r1 = dfx.Histo2D(("h_ee_thth","EE theta-theta R1;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r1_p4tht1","r1_p4tht2")
h_ee_thtE1_r2 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1 R2;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r2_p4tht1","r2_p4E1")
h_ee_thtE2_r2 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2 R2;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r2_p4tht2","r2_p4E2")
h_ee_thth_r2 = dfx.Histo2D(("h_ee_thth","EE theta-theta R2;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r2_p4tht1","r2_p4tht2")
h_ee_thtE1_r3 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1 R3;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r3_p4tht1","r3_p4E1")
h_ee_thtE2_r3 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2 R3;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r3_p4tht2","r3_p4E2")
h_ee_thth_r3 = dfx.Histo2D(("h_ee_thth","EE theta-theta R3;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r3_p4tht1","r3_p4tht2")

In [12]:
cc2 = R.TCanvas("cc2","cc2",1200,1200)
cc2.Divide(2,2)
pad1= cc2.cd(1)
# pad1.SetLogy()
h_ee_mass_r2.SetStats(0)
h_ee_mass_r2.SetLineColor(R.kOrange)
h_ee_mass_r2.SetLineWidth(2)
h_ee_mass_r2.Draw()
hx_ee_mass_r2.Scale(0.9)
hx_ee_mass_r2.SetStats(0)
hx_ee_mass_r2.SetLineColor(R.kRed)
hx_ee_mass_r2.SetLineWidth(2)
hx_ee_mass_r2.Draw("same")

h_ee_mass_r3.SetStats(0)
h_ee_mass_r3.SetLineColor(R.kBlue)
h_ee_mass_r3.SetLineWidth(2)
h_ee_mass_r3.Draw("same")
hx_ee_mass_r3.Scale(0.9)
hx_ee_mass_r3.SetStats(0)
hx_ee_mass_r3.SetLineColor(R.kRed+1)
hx_ee_mass_r3.SetLineWidth(2)
hx_ee_mass_r3.Draw("same")

# h_ee_mass_rx.SetStats(0)
# h_ee_mass_rx.SetLineColor(R.kOrange+2)
# h_ee_mass_rx.SetLineWidth(2)
# h_ee_mass_rx.Draw("same")
# hx_ee_mass_rx.Scale(0.9)
# hx_ee_mass_rx.SetStats(0)
# hx_ee_mass_rx.SetLineColor(R.kRed+2)
# hx_ee_mass_rx.SetLineWidth(2)
# hx_ee_mass_rx.Draw("same")

h_v0_mass_UC.SetStats(0)
h_v0_mass_UC.SetLineWidth(1)
h_v0_mass_UC.SetLineColor(R.kGreen+1)
h_v0_mass_UC.Draw("same")
h_v0_mass_BSC.SetLineWidth(1)
h_v0_mass_BSC.SetLineColor(R.kCyan)
h_v0_mass_BSC.Draw("same")
h_v0_mass_TC.SetLineColor(R.kGreen)
h_v0_mass_TC.Draw("same")

legend = R.TLegend(0.6,0.7,0.9,0.9)
legend.AddEntry(h_v0_mass_UC.GetPtr(),"Unconstrained Moller","l")
legend.AddEntry(h_v0_mass_BSC.GetPtr(),"Beamspot contrained Moller","l")
legend.AddEntry(h_v0_mass_TC.GetPtr(),"Target constrained Moller","l")
legend.AddEntry(h_ee_mass_r2.GetPtr(),"Moller r2","l")
legend.AddEntry(hx_ee_mass_r2.GetPtr(),"Not Moller r2","l")
legend.AddEntry(h_ee_mass_r3.GetPtr(),"Moller r3","l")
legend.AddEntry(hx_ee_mass_r3.GetPtr(),"Not Moller r3","l")
#legend.AddEntry(h_ee_mass_rx.GetPtr(),"Moller r-X","l")
#legend.AddEntry(hx_ee_mass_rx.GetPtr(),"Not Moller r-X","l")
legend.Draw()

pad2 = cc2.cd(2)
pad2.SetLogz()
h_ee_thth.SetStats(0)
h_ee_thth.Draw("colz")
l1 = R.TLine(0.,0.04,0.04,0.)
# l1.SetLineColor(R.kRed)
# l1.SetLineWidth(2)
# l1.Draw()
# l2 = R.TLine(0.,0.048,0.048,0.)
# l2.SetLineColor(R.kRed)
# l2.SetLineWidth(2)
# l2.Draw()
f1 = R.TF1("f1","2*asin(0.00011108696/sin(x/2))",0.0044,0.1)
f1.SetLineColor(R.kGreen)
f1.Draw("same")

pad3 = cc2.cd(3)
pad3.SetLogz()
h_ee_thtE1.SetStats(0)
h_ee_thtE1.Draw("colz")
f2 = R.TF1("f2","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f2.SetLineColor(R.kRed)
f2.Draw("same")

pad4 = cc2.cd(4)
#f1 = R.TF1("f1","2*asin(0.11108696/sin(x/2))",0.0001,0.1)
pad4.SetLogz()
h_ee_thtE2.SetStats(0)
h_ee_thtE2.Draw("colz")
f3 = R.TF1("f3","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f3.SetLineColor(R.kRed)
f3.Draw("same")


cc2.Draw()

In [13]:
h_phi_diff = dfx.Define("phi_diff","vector<double> out; for(int i=0; i< electron_pairs.size(); ++i){ out.push_back(p4v1[i].Phi()-p4v2[i].Phi());} return out;").Histo1D(("h_phi_diff","phi_diff",500,-1.5,1.5),"phi_diff")
h_phi_diff_r2 = dfx.Define("phi_diff","vector<double> out; for(int i=0; i< el_pairs_r2.size(); ++i){ double phi_diff = r2_p4v1[i].Phi()-r2_p4v2[i].Phi(); phi_diff = (phi_diff>0)?phi_diff: phi_diff+TMath::Pi(); out.push_back(phi_diff);} return out;").Histo1D(("h_phi_diff_r2","phi_diff_r2",500,-1.5,1.5),"phi_diff")
h_phi_diff_r3 = dfx.Define("phi_diff","vector<double> out; for(int i=0; i< el_pairs_r3.size(); ++i){ double phi_diff = r2_p4v1[i].Phi()-r2_p4v2[i].Phi(); phi_diff = (phi_diff>0)?phi_diff: phi_diff+TMath::Pi(); out.push_back(phi_diff);} return out;").Histo1D(("h_phi_diff_r2","phi_diff_r2",500,-1.5,1.5),"phi_diff")

In [14]:

cc3 = R.TCanvas("cc3","cc3",600,400)
#cc3.Divide(2,2)
#pad1= cc3.cd(1)
#   pad1.SetLogy()
h_phi_diff.SetStats(0)
h_phi_diff.Draw()
h_phi_diff_r2.SetLineColor(R.kRed)
h_phi_diff_r2.Draw("same")
h_phi_diff_r3.SetLineColor(R.kOrange)
h_phi_diff_r3.Draw("same")

cc3.Draw()


In [17]:
cc4 = R.TCanvas("cc4","cc4",1000,800)
#cc2.Divide(2,2)
#pad1= cc2.cd(1)
# pad1.SetLogy()
# h_ee_mass_r2.SetStats(0)
# h_ee_mass_r2.SetLineColor(R.kOrange)
# h_ee_mass_r2.SetLineWidth(2)
# h_ee_mass_r2.Draw()
# hx_ee_mass_r2.Scale(0.9)
# hx_ee_mass_r2.SetStats(0)
# hx_ee_mass_r2.SetLineColor(R.kRed)
# hx_ee_mass_r2.SetLineWidth(2)
# hx_ee_mass_r2.Draw("same")

h_ee_mass_r3.SetStats(0)
h_ee_mass_r3.SetLineColor(R.kBlue)
h_ee_mass_r3.SetLineWidth(2)
h_ee_mass_r3.Draw("same")
h_ee_mass_r3.Fit("gaus")

# h_ee_mass_rx.SetStats(0)
# h_ee_mass_rx.SetLineColor(R.kOrange)
# h_ee_mass_rx.SetLineWidth(2)
# h_ee_mass_rx.Draw("same")
# h_ee_mass_rx.Fit("gaus")


cc4.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       5746.5
NDf                       =          311
Edm                       =   6.2203e-08
NCalls                    =          138
Constant                  =      1351.11   +/-   9.03066     
Mean                      =    0.0486854   +/-   2.56961e-05 
Sigma                     =   0.00513437   +/-   2.39181e-05  	 (limited)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc4


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       5746.5
NDf                       =          311
Edm                       =  6.62031e-08
NCalls                    =          138
Constant                  =      1351.11   +/-   9.03066     
Mean                      =    0.0486854   +/-   2.56961e-05 
Sigma                     =   0.00513437   +/-   2.39181e-05  	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1946.38
NDf                       =           87
Edm                       =  9.35766e-08
NCalls                    =           78
Constant                  =      1258.54   +/-   9.8645      
Mean                      =    0.0478904   +/-   2.78809e-05 
Sigma                     =   0.00384941   +/-   2.05811e-05  	 (limited)


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       5746.5
NDf                       =          311
Edm                       =  6.62031e-08
NCalls                    =          138
Constant                  =      1351.11   +/-   9.03066     
Mean                      =    0.0486854   +/-   2.56961e-05 
Sigma                     =   0.00513437   +/-   2.39181e-05  	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1946.38
NDf                       =           87
Edm                       =  9.35766e-08
NCalls                    =           78
Constant                  =      1258.54   +/-   9.8645      
Mean                      =    0.0478904   +/-   2.78809e-05 
Sigma                     =   0.00384941   +/-   2.05811e-05  	 (limited)
